In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
#import os
#os.environ['CUDA_VISIBLE_DEVICES'] = "2"
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
from data_preprocess_functions import prepare_dataset,create_ziped_dataset1,buildDataSet,load_classes,get_train_test_dataset
from build_network import build_network,build_model
from utils import DrawPics,drawTriplets
from learning_utils import compute_probs,get_batch_hard,get_batch_random,compute_dist,compute_metrics,compute_interdist,draw_interdist,draw_roc,DrawTestImage
from keras.engine.topology import Layer
from sklearn.metrics import roc_curve,roc_auc_score
from keras import backend as K
from keras.optimizers import Adam
import time
import os
import os.path
from os import path
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
from PIL import Image
from keras.regularizers import l2
from keras import backend as K
from keras.utils import plot_model,normalize

class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)

    def triplet_loss(self, inputs):
        anchor, positive, negative = inputs
        p_dist = K.sum(K.square(anchor-positive), axis=-1)
        n_dist = K.sum(K.square(anchor-negative), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)

    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss
class NetworkConfig:
	def __init__(self, min_samples=0):
		self.min_samples = min_samples

networkConfig = NetworkConfig()
networkConfig.model_path = None
networkConfig.min_samples = 10
networkConfig.data_location = "./labeled"
networkConfig.extend = '.png'
networkConfig.prepare_dir = ''
networkConfig.prepare_num = 10
networkConfig.prepare_test_size = 20/ float(100)
output_dir="."
image_size = (256, 245 )
nb_classes=44

#def l2Norm(x):
 #       return  K.l2_normalize(x, axis=-1)
#def triplet_loss(y_true, y_pred):
 #   margin = K.constant(0.02)
  #  return K.mean(K.maximum(K.constant(0), K.square(y_pred[:,0,0]) - K.square(y_pred[:,1,0]) + margin))
#def accuracy(y_true, y_pred):
 #   return K.mean(y_pred[:,0,0] < y_pred[:,1,0])
#def euclidean_distance(vects):
 #   x, y = vects
  #  return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))

projectName = "plankton_recognition_LUT"
project_path = './{0}/'.format(projectName)
model_path = '../{0}/'.format(projectName)
if not path.exists(project_path):
    os.mkdir(project_path)
if not path.exists(model_path):
    os.mkdir(model_path)

BATCH_SIZE = 32
IMG_SIZE = (224, 224)
IMG_SHAPE = IMG_SIZE + (3,)
input_shape=IMG_SHAPE


In [ ]:
PATH="."
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'test')

train_dataset = image_dataset_from_directory(train_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
validation_dataset = image_dataset_from_directory(validation_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
class_names = train_dataset.class_names
class_names1 = validation_dataset.class_names
create_ziped_dataset1(train_dataset, validation_dataset)

In [ ]:
[dataset_train,dataset_test,x_train_origin,y_train_origin,x_test_origin,y_test_origin]=buildDataSet()
print("Checking shapes for class 0 (train) : ",dataset_train[0].shape)
print("Checking shapes for class 0 (test) : ",dataset_test[0].shape)
print("Checking first samples")
#for i in range(19,20):
    #DrawPics(dataset_train[i],5,template='Train {}',classnumber=i)
    #DrawPics(dataset_test[i],5,template='Test {}',classnumber=i)

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
from keras.applications import MobileNetV2
base_model = MobileNetV2(input_shape=IMG_SHAPE,include_top=False, weights='imagenet')
embeddingsize=21
net =base_model.output
net = tf.keras.layers.Flatten()(net) 
net = tf.keras.layers.Dropout(0.5)(net)
net = tf.keras.layers.Dense(4096, activation='relu', kernel_regularizer=l2(1e-3), kernel_initializer='he_uniform')(net)
net = tf.keras.layers.Dense(embeddingsize, activation=None, kernel_regularizer=l2(1e-3), kernel_initializer='he_uniform')(net)
net = tf.keras.layers.Flatten()(net) 

#Force the encoding to live on the d-dimentional hypershpere
net = tf.keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=-1))(net)
base_model =tf.keras.Model(base_model.input, net, name='tuned_model')
base_model.summary()


In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
base_model = VGG16(input_shape=IMG_SHAPE,weights='imagenet', include_top=False)
embeddingsize=44
net =base_model.output
net = tf.keras.layers.Dropout(0.5)(net)
net = tf.keras.layers.Flatten()(net) 
net = tf.keras.layers.Dense(4096, activation='relu', kernel_regularizer=l2(1e-2), kernel_initializer='he_uniform')(net)
net = tf.keras.layers.Dense(embeddingsize, activation=None, kernel_regularizer=l2(1e-2), kernel_initializer='he_uniform')(net)

#Force the encoding to live on the d-dimentional hypershpere
net = tf.keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=-1))(net)
base_model =tf.keras.Model(base_model.input, net, name='tuned_model')
base_model.summary()

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
base_model = ResNet50(weights='imagenet', include_top = False)
embeddingsize=21
net =base_model.output
net = tf.keras.layers.Dropout(0.5)(net)
net = tf.keras.layers.Dense(4096, activation='relu', kernel_regularizer=l2(1e-3), kernel_initializer='he_uniform')(net)
net = tf.keras.layers.Dense(embeddingsize, activation=None, kernel_regularizer=l2(1e-3), kernel_initializer='he_uniform')(net)
net = tf.keras.layers.Flatten()(net) 

#Force the encoding to live on the d-dimentional hypershpere
net = tf.keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=-1))(net)
base_model =tf.keras.Model(base_model.input, net, name='tuned_model')
base_model.summary()
   

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
base_model = VGG19(input_shape=IMG_SHAPE,weights='imagenet')
base_model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

embeddingsize=21
net =base_model.output
net = tf.keras.layers.Dropout(0.5)(net)
net = tf.keras.layers.Flatten()(net) 
net = tf.keras.layers.Dense(4096, activation='relu', kernel_regularizer=l2(1e-3), kernel_initializer='he_uniform')(net)
net = tf.keras.layers.Dense(embeddingsize, activation=None, kernel_regularizer=l2(1e-3), kernel_initializer='he_uniform')(net)

#Force the encoding to live on the d-dimentional hypershpere
net = tf.keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=-1))(net)
base_model =tf.keras.Model(base_model.input, net, name='tuned_model')
base_model.summary()
   

    

In [ ]:
base_model.trainable = True
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))
# Fine-tune from this layer onwards
fine_tune_at = 10
# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
hardtriplets = get_batch_hard(50,1,1, base_model, dataset_train)
print("Shapes in the hardbatch A:{0} P:{1} N:{2}".format(hardtriplets[0].shape, hardtriplets[1].shape, hardtriplets[2].shape))
#drawTriplets(hardtriplets)

In [ ]:

model = build_model(input_shape,base_model)
optimizer = Adam(lr = 0.00006)
model.compile(loss=None,optimizer=optimizer)
#model.compile(optimizer="rmsprop", loss=triplet_loss, metrics=[accuracy])
model.summary()
#plot_model(network_train,show_shapes=True, show_layer_names=True, to_file='02 model.png')
print(model.metrics_names)
n_iteration=0
#network_train.load_weights('mnist-160k_weights.h5')


In [ ]:
evaluate_every = 50 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 800# No. of training iterations
n_val = 250 # how many one- shot tasks to validate on
log_every = 50
#####Testing on an untrained network
probs,yprob = compute_probs(base_model,x_test_origin[:44,:,:,:],y_test_origin[:44])
fpr, tpr, thresholds,auc = compute_metrics(probs,yprob)
draw_roc(fpr, tpr,thresholds,auc)
draw_interdist(base_model,0,dataset_test)

In [ ]:
# Trainning of the model 
def prediction1(network, x_test_origin, dataset_test, test_label,top,threshold,classindicator=-1, refidx=1,nb_test_class=21 ):
    predicted=[]
    _, w,h,c = dataset_test[0].shape
    nbimages=x_test_origin.shape[0]
    nb_display=21
    # planton spieces/ classes 
    classes = load_classes(networkConfig.data_location, networkConfig.extend, networkConfig.min_samples)
    #generates embedings for reference images
    ref_images = np.zeros((nb_test_class,w,h,c))
    for i in range(nb_test_class):
        ref_images[i,:,:,:] = dataset_test[i][refidx,:,:,:]
    ref_embedings = network.predict(ref_images)   
    c=0
    for i in range(0,nbimages):
        #generates embedings for given images
        image_embedings = network.predict(np.expand_dims(x_test_origin[i,:,:,:],axis=0))
        if nbimages>1:
            trueclass=i
        else:
            trueclass=classindicator
        distdtype=[('class', int), ('dist', float)]
        dist = np.zeros(nb_test_class, dtype=distdtype)        
        #Compute distances
        for ref in range(nb_test_class):
            #Compute distance between this images and references
            dist[ref] = (ref,compute_dist(image_embedings[:],ref_embedings[ref,:]))          
        #sort
        sorted_dist = np.sort(dist, order='dist')
        found=False

        for a in range(0,top):
            if (test_label[i]==classes[sorted_dist['class'][a]]):
                predicted.append(classes[sorted_dist['class'][a]])
                found=True;
                c=c+1
                break;
        if(found==False):
            predicted.append(classes[sorted_dist['class'][0]])
    
    print(c)        
    return predicted

In [ ]:
def test_images(network, images, dataset_test,indx, threshold, classindicator=-1, refidx=0,nb_test_class=21 ):
    predicted=[]
    _, w,h,c = dataset_test[0].shape
    nbimages=images.shape[0]
    nb_display=21
    #generates embedings for given images
    image_embedings = network.predict(images)
    # planton spieces/ classes 
    classes = load_classes(networkConfig.data_location, networkConfig.extend, networkConfig.min_samples)
    #generates embedings for reference images
    ref_images = np.zeros((nb_test_class,w,h,c))
    for i in range(nb_test_class):
        ref_images[i,:,:,:] = dataset_test[i][refidx,:,:,:]
    ref_embedings = network.predict(ref_images)
        
    for i in range(nbimages):
        if nbimages>1:
            trueclass=i
        else:
            trueclass=classindicator
        
        #Prepare the figure
        fig=plt.figure(figsize=(50,50))
        subplot = fig.add_subplot(1,nb_display+1,1)
        subplot.axis("off")
        plotidx = 2
            
        #Draw this image    
        plt.imshow(images[i,:,:,0])
        subplot.title.set_text("Test image")
            
        distdtype=[('class', int), ('dist', float)]
        dist = np.zeros(nb_test_class, dtype=distdtype)
        
        #Compute distances
        for ref in range(nb_test_class):
            #Compute distance between this images and references
            dist[ref] = (ref,compute_dist(image_embedings[i,:],ref_embedings[ref,:]))
            
        #sort
        sorted_dist = np.sort(dist, order='dist')
        predicted.append(classes[sorted_dist['class'][0]])
        #Draw
        for j in range(min(21,nb_test_class)):
            subplot = fig.add_subplot(1,nb_display+1,plotidx)
            plt.imshow(ref_images[sorted_dist['class'][j],:,:,0])
            subplot.axis("off")
            #Red for sample above threshold
            if (sorted_dist['dist'][j] > threshold):
                if (trueclass == sorted_dist['class'][j]):
                    color = (1,0,0)
                    label = "TRUE"
                else:
                    color = (0.5,0,0)
                    label = "Class {0}".format(sorted_dist['class'][j])
            else:
                if (trueclass == sorted_dist['class'][j]):
                    color = (0, 1, 0)
                    label = "TRUE"
                else:
                    color = (0, .5, 0)
                    label = "Class {0}".format(sorted_dist['class'][j])
                
            subplot.set_title("{0}\n{1:.3e}".format(label,sorted_dist['dist'][j]),color=color)
            plotidx += 1
    return predicted
 

In [ ]:
evaluate_every = 100 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 5000# No. of training iterations
n_val = 100 # how many one- shot tasks to validate on
## # #
t_start = time.time()
n_iteration=0
file = open("store.txt", "w")
for i in range(1, n_iter+1):
    #triplets = get_batch_random(batch_size,dataset_train)
    triplets=get_batch_hard(200,1,1,base_model,dataset_train)
    loss = model.train_on_batch(triplets, None)
    #network_train.save_weights(file)
    n_iteration += 1
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("[{3}] Time for {0} iterations: {1:.1f} mins, Train Loss: {2}".format(i, (time.time()-t_start)/60.0,loss,n_iteration))
        probs,yprob = compute_probs(base_model,x_test_origin[:500,:,:,:],y_test_origin[:500])
        #fpr, tpr, thresholds,auc = compute_metrics(probs,yprob)
        #draw_roc(fpr, tpr,thresholds,auc)
        #draw_interdist(base_model,0,dataset_test)
    if i % 1000 == 0:
        probs,yprob = compute_probs(base_model,x_test_origin[:500,:,:,:],y_test_origin[:500])
        fpr, tpr, thresholds,auc = compute_metrics(probs,yprob)
        draw_roc(fpr, tpr,thresholds,auc)
        top=1
        predicted=prediction1(base_model, x_test_origin, dataset_test,y_test_origin,top, threshold=abs(.007))
        Top1_count=np.sum(predicted == y_test_origin)
        print(Top1_count)
        #base_model.save_weights('{1}3x-temp_weights_{0:08d}.h5'.format(n_iteration,model_path))    
#Final save
#model.save_weights('{1}3x-temp_weights_{0:08d}.h5'.format(n_iteration,model_path)) 
ind=0
predicted=[]
for i in range(1,4):
    predicted.append(test_images(base_model, np.expand_dims(x_test_origin[i,:,:,:],axis=0), dataset_test,i,threshold=abs(1)))
    ind=ind+ind

top=1
predicted=prediction1(base_model, x_test_origin, dataset_test,y_test_origin,top, threshold=abs(.007))
Top1_count=np.sum(predicted == y_test_origin)

top=2
predicted=prediction1(base_model, x_test_origin, dataset_test,y_test_origin,top, threshold=abs(1))
Top2_count=np.sum(predicted == y_test_origin)

top=3
predicted=prediction1(base_model, x_test_origin, dataset_test,y_test_origin,top, threshold=abs(1))
Top3_count=np.sum(predicted == y_test_origin)

top=4
predicted=prediction1(base_model, x_test_origin, dataset_test,y_test_origin,top, threshold=abs(1))
Top4_count=np.sum(predicted == y_test_origin)

top=5
predicted=prediction1(base_model, x_test_origin, dataset_test,y_test_origin,top, threshold=abs(1))
Top5_count=np.sum(predicted == y_test_origin)

import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

objects = ('Top1', 'Top2', 'Top3', 'Top4', 'Top5')
y_pos = np.arange(len(objects))
performance = [Top1_count,Top2_count,Top3_count,Top4_count,Top5_count]

bar1=plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Number of correct classified species')
plt.title('Top accuracy')
# Add counts above the two bar graphs
for rect in bar1 :
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width()/2.0, height, '%d ' % ((height*100)/63), ha='center', va='bottom')
plt.show()

import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
confusion_matrix=confusion_matrix(y_test_origin,predicted)
class_names = load_classes(networkConfig.data_location, networkConfig.extend, networkConfig.min_samples)
df_cm = pd.DataFrame(confusion_matrix, index = [i for i in class_names],
              columns = [i for i in class_names])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True,cmap="OrRd")

file = open("store.txt", "w")
file.write(" %s," %( predicted[:]))


In [ ]:
#Full evaluation
probs,yprob = compute_probs(base_model,x_test_origin,y_test_origin)
fpr, tpr, thresholds,auc = compute_metrics(probs,yprob)
draw_roc(fpr, tpr,thresholds,auc)
draw_interdist(base_model,n_iter,dataset_test)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn')
# plot roc curves
plt.plot(fpr, tpr, linestyle='--',color='orange', label='triploss learning')
# title
plt.title('ROC curve')
# x label
plt.xlabel('False Positive Rate')
# y label
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('ROC1',dpi=300)
plt.show();

In [ ]:
ind=0
predicted=[]
for i in range(0,6):
    predicted.append(test_images(base_model, np.expand_dims(x_test_origin[i,:,:,:],axis=0), dataset_test,i,threshold=abs(0.2716)))
    ind=ind+ind

In [ ]:


def buildDataSet1():
    # prepare dataset  for train and test
    PATH="/content/drive/MyDrive/plankton_recognition"
    
	train_dir = os.path.join(PATH, 'train')
	validation_dir = os.path.join(PATH, 'test')
	BATCH_SIZE = 1050
	IMG_SIZE = (224, 224)
	train_dataset = image_dataset_from_directory(train_dir,
	                                             shuffle=False,
	                                            batch_size=BATCH_SIZE,
	                                             image_size=IMG_SIZE)
	validation_dataset = image_dataset_from_directory(validation_dir,
	                                                  shuffle=False,
	                                                  batch_size=BATCH_SIZE,
	                                                  image_size=IMG_SIZE)
	
	# train data
	for images, labels in train_dataset.take(1):
        x_train_origin=images
        y_train_origin=labels
	# test data
	for images, labels in validation_dataset.take(1):
        x_test_origin=images
        y_test_origin=labels
	
	dataset_train = []
	dataset_test = []
	#Sorting images by classes and normalize values 0=>1
	n_classes=21
	classes = load_classes(networkConfig.data_location, networkConfig.extend, networkConfig.min_samples)
	for n in range(0,n_classes):
        images_class_n = np.asarray([row for idx,row in enumerate(x_train_origin) if class_names[y_train_origin[idx]]==classes[n] ])
        dataset_train.append(images_class_n/255)
        images_class_n = np.asarray([row for idx,row in enumerate(x_test_origin) if class_names[y_test_origin[idx]]==classes[n]  ])
        dataset_test.append(images_class_n/255)
    return dataset_train,dataset_test,x_train_origin,y_train_origin,x_test_origin,y_test_origin


In [ ]:
def prediction1(network, x_test_origin, dataset_train, test_label,top,threshold,classindicator=-1, refidx=1 ):
    predicted=[]
    _, w,h,c = dataset_test[0].shape
    nbimages=x_test_origin.shape[0]
    nb_test_class=len(dataset_train)
    # planton spieces/ classes 
    classes = load_classes(networkConfig.data_location, networkConfig.extend, networkConfig.min_samples)
    nb_display=len(classes)
    #generates embedings for reference images
    ref_images = np.zeros((nb_test_class,w,h,c))
    for i in range(nb_test_class):
        ref_images[i,:,:,:] = dataset_train[i][refidx,:,:,:]
    ref_embedings = network.predict(ref_images) 
    
    for i in range(nbimages):
        #generates embedings for given images
        image_embedings = network.predict(np.expand_dims(x_test_origin[i,:,:,:],axis=0))
        distdtype=[('class', int), ('dist', float)]
        dist = np.zeros(nb_test_class, dtype=distdtype)        
        #Compute distances
        for ref in range(nb_test_class):
            #Compute distance between this images and references
            dist[ref] = (ref,compute_dist(image_embedings[:],ref_embedings[ref,:]))  
            #dist[ref] = (ref,euclidean_distance(image_embedings[:],ref_embedings[ref,:]))
        #sort
        sorted_dist = np.sort(dist, order='dist')
        found=False

        for a in range(top):
            if (test_label[i]==classes[sorted_dist['class'][a]]):
                predicted.append(classes[sorted_dist['class'][a]])
                found=True;
                c=c+1
                break;
        if(found==False):
            predicted.append(classes[sorted_dist['class'][0]])
    
    print(c)        
    return predicted

In [ ]:
print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
import datetime
for i in range(1, n_iter+1):
    microtask_start =time.time() 
    triplets=get_batch_hard1(200,1,1,base_model,dataset_train)
    #triplets,quadruplets = get_batch_hardOptimized(100,16,16,network3,network4,metric_network4, dataset_train)
    timetogetbatch = time.time()-microtask_start
    microtask_start =time.time() 
    loss1 = model.train_on_batch(triplets, None)
    timebatch3 = time.time()-microtask_start
    n_iteration += 1
    if i % log_every == 0:
        wandb.log({'loss3x': loss1,'loss4x': loss2}, step=n_iteration)
    if i % evaluate_every == 0:
        elapsed_minutes = (time.time()-t_start)/60.0
        rate = i/elapsed_minutes 
        #eta = datetime.now()+timedelta(minutes=(n_iter-i)/rate)
        #eta = eta + timedelta(hours=3) #finnish time
        print("[{3}] Time for {0} iterations: {1:.1f} mins, Train Loss: {2}".format(i, (time.time()-t_start)/60.0,loss,n_iteration))
        base_model.save_weights('{1}3x-temp_weights_{0:08d}.h5'.format(n_iteration,model_path))    
#Final save
network3_train.save_weights('{1}3x-temp_weights_{0:08d}.h5'.format(n_iteration,model_path))  
print("Done !")

In [ ]:
def test_images1(network, images, dataset_test,indx, threshold, classindicator=-1, refidx=0,nb_test_class=21 ):
    _, w,h,c = dataset_test[0].shape
    nbimages=images.shape[0]
    nb_display=21
    #generates embedings for given images
    image_embedings = network.predict(images)
    # planton spieces/ classes 
    classes = load_classes(networkConfig.data_location, networkConfig.extend, networkConfig.min_samples)
    #generates embedings for reference images
    ref_images = np.zeros((nb_test_class,w,h,c))
    for i in range(nb_test_class):
        ref_images[i,:,:,:] = dataset_test[i][refidx,:,:,:]
    ref_embedings = network.predict(ref_images)
            
    for i in range(nbimages):
        if nbimages>1:
            trueclass=i
        else:
            trueclass=classindicator
        
        #Prepare the figure
        fig=plt.figure(figsize=(50,50))
        subplot = fig.add_subplot(1,nb_display+1,1)
        subplot.axis("off")
        plotidx = 2
            
        #Draw this image    
        plt.imshow(images[i,:,:,0])
        subplot.title.set_text("Test image")
            
        distdtype=[('class', int), ('dist', float)]
        dist = np.zeros(nb_test_class, dtype=distdtype)
        
        #Compute distances
        for ref in range(nb_test_class):
            #Compute distance between this images and references
            dist[ref] = (ref,compute_dist(image_embedings[i,:],ref_embedings[ref,:]))
            
        #sort
        sorted_dist = np.sort(dist, order='dist')
        
        #Draw
        for j in range(min(21,nb_test_class)):
            subplot = fig.add_subplot(1,nb_display+1,plotidx)
            plt.imshow(ref_images[sorted_dist['class'][j],:,:,0])
            subplot.axis("off")
            #Red for sample above threshold
            if (sorted_dist['dist'][j] > threshold):
                if (trueclass == sorted_dist['class'][j]):
                    color = (1,0,0)
                    label = "TRUE"
                else:
                    color = (0.5,0,0)
                    label = "Class {0}".format(sorted_dist['class'][j])
            else:
                if (trueclass == sorted_dist['class'][j]):
                    color = (0, 1, 0)
                    label = "TRUE"
                else:
                    color = (0, .5, 0)
                    label = "Class {0}".format(sorted_dist['class'][j])
                
            subplot.set_title("{0}\n{1:.3e}".format(label,sorted_dist['dist'][j]),color=color)
            plotidx += 1   

'''
    Evaluate some pictures vs some samples in the test set
        image must be of shape(1,w,h,c)
    Returns
        scores : resultat des scores de similarités avec les images de base => (N)
'''
def DrawTestImage(network, images, dataset_test, index,refidx=0,nb_test_class=21):
    N=4
    _, w,h,c = dataset_test[0].shape
    nbimages=images.shape[0]
    classes = load_classes(networkConfig.data_location, networkConfig.extend, networkConfig.min_samples)
    #generates embedings for given images
    image_embedings = network.predict(images)
    
    #generates embedings for reference images
    ref_images = np.zeros((nb_classes,w,h,c))
    for i in range(nb_classes):
        ref_images[i,:,:,:] = dataset_test[i][refidx,:,:,:]
    ref_embedings = network.predict(ref_images)
            
    for i in range(nbimages):
        #Prepare the figure
        fig=plt.figure(figsize=(50,50))
        subplot = fig.add_subplot(1,nb_classes+1,1)
        subplot.axis("off")
        plotidx = 2
            
        #Draw this image    
        plt.imshow(images[i,:,:,0],vmin=0, vmax=1,cmap='Greys')
        subplot.title.set_text(classes[index])
            
        for ref in range(nb_classes):
            #Compute distance between this images and references
            dist = compute_dist(image_embedings[i,:],ref_embedings[ref,:])
            #Draw
            subplot = fig.add_subplot(1,nb_classes+1,plotidx)
            subplot.axis("off")
            plt.imshow(ref_images[ref,:,:,0])
            subplot.title.set_text(("Class {0}\n{1:.3e}".format(ref,dist)))
            plotidx += 1
    
# Trainning of the model 

def prediction1(network, x_test_origin, dataset_test, test_label,top,threshold,classindicator=-1, refidx=0,nb_test_class=50 ):
    predicted=[]
    _, w,h,c = dataset_test[0].shape
    nbimages=x_test_origin.shape[0]
    nb_display=50
    # planton spieces/ classes 
    classes = load_classes(networkConfig.data_location, networkConfig.extend, networkConfig.min_samples)
    nb_display=len(classes)
    nb_test_class=len(classes)
    #generates embedings for reference images
    ref_images = np.zeros((nb_test_class,w,h,c))
    for i in range(nb_test_class):
        ref_images[i,:,:,:] = dataset_test[i][refidx,:,:,:]
    ref_embedings = network.predict(ref_images)   
    c=0
    for i in range(0,nbimages):
        #generates embedings for given images
        image_embedings = network.predict(np.expand_dims(x_test_origin[i,:,:,:],axis=0))
        if nbimages>1:
            trueclass=i
        else:
            trueclass=classindicator
        distdtype=[('class', int), ('dist', float)]
        dist = np.zeros(nb_test_class, dtype=distdtype)        
        #Compute distances
        for ref in range(nb_test_class):
            #Compute distance between this images and references
            dist[ref] = (ref,compute_dist(image_embedings[:],ref_embedings[ref,:]))          
        #sort
        sorted_dist = np.sort(dist, order='dist')
        found=False

        for a in range(0,top):
            if (test_label[i]==classes[sorted_dist['class'][a]]):
                predicted.append(classes[sorted_dist['class'][a]])
                found=True;
                c=c+1
                break;
        if(found==False):
            predicted.append(classes[sorted_dist['class'][0]])
    print(c)        
    return predicted    

    

In [ ]:
correct=[]
time_taken=[]
for k in range(1,6):
    topac=[]
    timeacc=[]
    for i in range(1,7):
        [predicted,time_t]=classifier(base_model, x_test_origin, dataset_train,y_test_origin,i,k)
        count=np.sum(predicted == y_test_origin)
        topac.append(count)
        timeacc.append(time_t)
    correct.append(topac)
    time_taken.append(timeacc)

In [ ]:
def classifier(network, x_test_origin, dataset_train,test_label,refidx,top):
    t_start = time.time()
    predicted=[]
    _, w,h,c = dataset_train[0].shape
    nbimages=x_test_origin.shape[0]
    # planton spieces/ classes 
    classes = load_classes(networkConfig.data_location, networkConfig.extend, networkConfig.min_samples)
    nb_display=len(classes)
    nb_test_class=len(dataset_train)
    for i in range(0,nbimages):
        distdtype=[('class', int), ('dist', float)]
        dist1 = np.zeros(refidx, dtype=distdtype)
        #generates embedings for given images
        c=0;
        image_embedings = network.predict(np.expand_dims(x_test_origin[i,:,:,:],axis=0))
        for k in range(0,refidx):
            ref_images = np.zeros((nb_test_class,w,h,3))
            for j in range(0,nb_test_class):
                ref_images[j,:,:,:] = dataset_train[j][k,:,:,:]
                
            ref_embedings = network.predict(ref_images)  
            #Compute distances
            distdtype=[('class', int), ('dist', float)]
            dist = np.zeros(nb_test_class, dtype=distdtype)
            for ref in range(0,nb_test_class):
                #Compute distance between this images and references
                dist[ref] = (ref,compute_dist(image_embedings[:],ref_embedings[ref,:]))
                #dist[ref] = (ref,euclidean_distance(image_embedings[:],ref_embedings[ref,:]))
            #sort
            sorted_dist = np.sort(dist, order='dist')
            found=False
            for a in range(0,top):
                if (test_label[i]==classes[sorted_dist['class'][a]]):
                    predicted.append(classes[sorted_dist['class'][a]])
                    found=True;
                    c=c+1
                    break;
            if(found==True):
                break;
            else:
                dist1[k]=(sorted_dist['class'][0],sorted_dist['dist'][0])
        if(c==0):
            sorted_dist1 = np.sort(dist1, order='dist')
            predicted.append(classes[sorted_dist1['class'][0]])
    timeT=((time.time()-t_start)/60.0)
    return predicted, timeT  